In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import random
import numpy as np
import util
from pynput.mouse import Button, Controller

# Initialize mouse controller
mouse = Controller()

# screen size for mouse movement
screen_width, screen_height = pyautogui.size()

# Initialize MediaPipe hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=1
)

mp_draw = mp.solutions.drawing_utils

def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]  # Assuming only one hand is detected
        index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
        return index_finger_tip
    return None, None

def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_width)
        y = int(index_finger_tip.y / 2 * screen_height)  # Adjusting y-coordinate
        pyautogui.moveTo(x, y)
        
def is_left_click(landmark_list, thumb_index_dist):
    return (
            util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
            util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) > 90 and
            thumb_index_dist > 50
    )

def is_right_click(landmark_list, thumb_index_dist):
    return (
            util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
            util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) > 90  and
            thumb_index_dist > 50
    )
    
def is_double_click(landmark_list, thumb_index_dist):
    return (
            util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
            util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
            thumb_index_dist > 50
    )

def get_angle(p1, p2, p3):
    angle = np.degrees(
        np.arctan2(p3[1] - p2[1], p3[0] - p2[0]) - np.arctan2(p1[1] - p2[1], p1[0] - p2[0])
    )
    return abs(angle)

def is_closed_fist(landmarks_list):
    return all(get_angle(landmarks_list[i], landmarks_list[i + 1], landmarks_list[i + 2]) < 30 for i in [5, 9, 13, 17])

def is_two_fingers_up(landmarks_list):
    return (get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) > 160 and 
            get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) > 160 and
            all(get_angle(landmarks_list[i], landmarks_list[i + 1], landmarks_list[i + 2]) < 30 for i in [13, 17]))

def is_open_palm(landmarks_list):
    return all(get_angle(landmarks_list[i], landmarks_list[i + 1], landmarks_list[i + 2]) > 160 for i in [5, 9, 13, 17])

def detect_gesture(frame, landmark_list, processed):
    if len(landmark_list) >= 21:
        index_finger_tip = find_finger_tip(processed)
        
        # Move mouse
        move_mouse(index_finger_tip)
        
        # Gesture detection for cut, copy, paste
        if is_two_fingers_up(landmark_list):
            pyautogui.hotkey('ctrl', 'x')
            cv2.putText(frame, "Cut", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        elif is_closed_fist(landmark_list):
            pyautogui.hotkey('ctrl', 'c')
            cv2.putText(frame, "Copy", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        elif is_open_palm(landmark_list):
            pyautogui.hotkey('ctrl', 'v')
            cv2.putText(frame, "Paste", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

def main():
    cap = cv2.VideoCapture(0)
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)  # Flip for mirror effect
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            processed = hands.process(frameRGB)

            landmark_list = []
            if processed.multi_hand_landmarks:
                hand_landmarks = processed.multi_hand_landmarks[0]  # Assuming only one hand is detected
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                for lm in hand_landmarks.landmark:
                    landmark_list.append((lm.x, lm.y))

            detect_gesture(frame, landmark_list, processed)

            cv2.imshow('Frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    main()
